# RL model voor 2x2x2 yard
In deze notebook wordt een RL model gemaakt voor een container yard van 3 bij 3. Het gaat hier om een indeling waar 9 containers van 3 verschillende zeevaartschepen (die ze op komen halen) geplaatst moeten worden. Er zijn 3 containers per zeevaartschip, wat betekent dat de optimale oplossing simpel te bedenken en te checken is: Alle containers moeten dan in 1 keer gepakt kunnen worden, zoder verplaatsingen.

### Imports

In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import random
import pandas as pd
import numpy as np
import random
import os

from IPython.display import display
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack,DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

### Functie matrix --> state

In [2]:
def create_matrix(x):
    data = {}
    for i in range(0,x):
        data[i] = [0]*x
    nullen_matrix = pd.DataFrame(data)
    return nullen_matrix

In [3]:
create_matrix(3)

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


### Functie voor reward
Hierin worden penalty's gegeven voor slechte of niet toegestane zetten en rewards voor goede/optimale zetten. Per zet worden deze uitgedeeld, en het totaal levert de score van die 'game' op. Langzaam zou de reward steeds hoger moeten worden tijdens het trainen van het RL model.

In [4]:
# + als die m naast zelfde containernummer plaatst
def reward_functie(matrix, breedte, plek, container, y_row, x_col):
    reward = 0
    
    #op lege plek plaatsen, + of -
    if plek != 0:
        reward -= 1
    
    #container naast zelfde, +
    if x_col > 0 and matrix.iloc[y_row, x_col-1] == container:
        reward += 1
    if x_col < (breedte-1) and matrix.iloc[y_row, x_col+1] == container:
        reward += 1
    
    #container tussen 2 in plaatsen, -
    if x_col != (0 or (breedte-1)) and (matrix.iloc[y_row, x_col-1] and matrix.iloc[y_row, x_col+1]) != 0:
        reward -= 1
    
    return reward

### Environment maken
Er is hier gewerkt met als state/observation space een matrix met 3 rijen en kolommen. Links en rechts moeten de lange zijes van de containers weergeven.

In [5]:
class Mijn_Env(Env):
    def __init__(self, breedte_matrix):
        self.x = breedte_matrix
        
        self.action_space = MultiDiscrete([self.x,self.x], dtype=int)
        self.observation_space = Dict({'matrix':Box(low=int(0),high=int(9),shape=(self.x,self.x), dtype=int), 'container':Discrete(4)})
        
        self.state = create_matrix(self.x) #achteraf naar numpy
        
        # nummers staan voor zeevaartschip waar containers voor bestemd zijn. Komen random volgorde binnen (want volgorde is onzeker)
        self.list_containers = list([1,1,1,2,2,2,3,3,3])
        random.shuffle(self.list_containers)
        
        # 9 te plaatsen containers
        self.duration = 9
        
    def step(self, action):
        self.duration -= 1
        container = self.list_containers.pop()
        y_row = action[0]
        x_col = action[1]
        plek = self.state.iloc[y_row,x_col]
        
        # REWARD
        reward = reward_functie(self.state, self.x, plek, container, y_row, x_col)
        # PLAATSEN
        self.state.iloc[y_row, x_col] = container
        # REWARD
        #rij dezelfde compleet maken, + 
        if self.state.iloc[y_row, 0:self.x].to_list() == ([1]*self.x or [2]*self.x or [3]*self.x):
            reward += 2
        
        # STOPPEN
        if self.duration <= 0:
            done=True
            obs = {'container':np.nan, 'matrix':self.state.to_numpy()}
        else:
            done=False
            # geef mee aan model in obs: volgende te plaatsen container + huidige state van yard
            obs = {'container':self.list_containers[-1], 'matrix':self.state.to_numpy()}
            
        info = {}
        
        return obs, reward, done, info
    
    def render(self, mode = 'human'):
        display(self.state)
    
    def reset(self):
        self.duration = 9
        # terug naar lege matrix:
        self.state = create_matrix(self.x)
        # nieuwe random list_containers
        self.list_containers = list([1,1,1,2,2,2,3,3,3])
        random.shuffle(self.list_containers)

        obs = {'container':self.list_containers[-1], 'matrix':self.state.to_numpy()}
        return obs

In [6]:
env = Mijn_Env(3)

In [7]:
env.state

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


In [8]:
print(env.list_containers)

[2, 1, 3, 2, 2, 3, 1, 3, 1]


### Environment testen

In [9]:
episodes = 2
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


,0,1,2
0,0,0,0
1,0,2,0
2,0,0,0


,0,1,2
0,0,0,0
1,0,3,0
2,0,0,0


,0,1,2
0,0,0,0
1,3,3,0
2,0,0,0


,0,1,2
0,0,0,0
1,3,3,0
2,2,0,0


...

,0,1,2
0,2,0,0
1,0,2,3
2,3,0,0


,0,1,2
0,2,0,0
1,1,2,3
2,3,0,0


,0,1,2
0,2,0,0
1,1,2,3
2,1,0,0


,0,1,2
0,1,0,0
1,1,2,3
2,1,0,0


Episode:2 Score:-5


### Environment trainen

In [10]:
model = PPO("MultiInputPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/opt/jupyterhub/anaconda/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [11]:
model.learn(total_timesteps=100000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 9        |
|    ep_rew_mean     | -2.74    |
| time/              |          |
|    fps             | 1014     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9           |
|    ep_rew_mean          | -2.63       |
| time/                   |             |
|    fps                  | 791         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013572126 |
|    clip_fraction        | 0.163       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.19       |
|    explained_variance   | -0.0865     |
|    learning_rate        | 0.

### Model gebruiken/evalueren

In [12]:
evaluate_policy(model , env, n_eval_episodes=1, return_episode_rewards = True, render=True)

/opt/jupyterhub/anaconda/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


,0,1,2
0,2,0,0
1,0,0,0
2,0,0,0


,0,1,2
0,2,2,0
1,0,0,0
2,0,0,0


,0,1,2
0,2,2,0
1,0,0,0
2,0,1,0


,0,1,2
0,2,2,0
1,0,0,0
2,1,1,0


...

,0,1,2
0,2,2,0
1,3,3,0
2,1,1,0


,0,1,2
0,2,2,0
1,3,3,3
2,1,1,0


,0,1,2
0,2,2,2
1,3,3,3
2,1,1,0


,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


([8.0], [9])

# Conclusie
In het testvoorbeeld is te zien dat de optimale indeling gevonden is. Er staan in elke rij 3 containers voor zelfde schip, dus het maakt niet uit welk schip nu aankomt: de stacker kan bij de conatainers zonder dat die andere containers hoeft de verplaatsen. 